In [ ]:
# 라이브러리 로드 
import pandas as pd
from database import MyDB

In [ ]:
# 로컬 DB server와의 연결 
db1 = MyDB()

In [ ]:
query_1 = """
    select * from `emp`
"""
emp_df = pd.DataFrame(db1.sql_query(query_1))
emp_df

In [ ]:
query_2 = """
    select * from `dept`
"""
dept_df = pd.DataFrame(db1.sql_query(query_2))
dept_df

In [ ]:
emp_df.head(1)

In [ ]:
# emp table에서 SAL 컬럼의 데이터가 1500 이상인 사원의 정보를 확인
# DataFrame을 이용하여 확인 
# 인덱스의 조건식 -> 기준이 인덱스의 값, value는 bool 형태의 데이터(Series)
flag = emp_df['SAL'] >= 1500
# 인덱스의 조건식을 이용하여 데이터프레임을 필터링 
    # loc[인덱스의 조건, 컬럼의 조건]
emp_df.loc[flag, ]

In [ ]:
# select query문을 이용하여 SAL이 1500이상인 데이터를 확인 
query_3 = """
    select * from `emp`
    where `SAL` >= 1500
"""
pd.DataFrame(
    db1.sql_query(query_3)
)

In [ ]:
# emp table에서 SAL이 1500 이상 이고 JOB이 MANAGER 인 사원의 정보를 확인 
query_4 = """
    select * from `emp`
    where 
    `SAL` >= 1500
    and
    `JOB` = "MANAGER"
"""
pd.DataFrame(
    db1.sql_query(query_4)
)

In [ ]:
# 위의 조건식을 이용하여 같은 결과를 DataFrame에서 출력하시오
flag = emp_df['SAL'] >= 1500
flag2 = emp_df['JOB'] == 'MANAGER'
emp_df.loc[ flag & flag2 , ]

In [ ]:
emp_df.loc[
    (emp_df['SAL'] >= 1500) & (emp_df['JOB'] == 'MANAGER') ,
]

In [ ]:
emp_df.query(
    "(SAL >= 1500) and (JOB == 'MANAGER')"
)

In [ ]:
# SAL의 데이터가 1500이상이고 2500이하인 사원번호와 사원 이름을 출력
query_5 = """
    select `EMPNO`, `ENAME` from `emp`
    where 
    `SAL` >= 1500 
    and 
    `SAL` <= 2500
"""
pd.DataFrame(
    db1.sql_query(query_5)
)

In [ ]:
query_6 = """
    select `EMPNO`, `ENAME` from `emp`
    where 
    `SAL` between 1500 and 2500
"""
pd.DataFrame(
    db1.sql_query(query_6)
)

In [ ]:
# emp_df를 이용하여 위와 같은 결과를 출력하시오
emp_df.loc[
    (emp_df['SAL'] >= 1500)
    & 
    (emp_df['SAL'] <= 2500) , 
    ['EMPNO', 'ENAME']
]

In [ ]:
emp_df.loc[
    emp_df['SAL'].isin(range(1500, 2501)), 
    ['EMPNO', 'ENAME']
]

In [ ]:
emp_df.loc[
    emp_df['SAL'].between(1500, 2500), 
    ['EMPNO', 'ENAME']
]

In [ ]:
# sql query 조건에서 2개의 데이터중 하나가 포함되어있는 조건 
# JOB의 데이터가 MANAGER이거나 SALESMAN인 사원의 정보를 출력 
query_7 = """
    select * from `emp`
    where 
    `JOB` = "MANAGER"
    or
    `JOB` = "SALESMAN"
"""
pd.DataFrame(
    db1.sql_query(query_7)
)

In [ ]:
query_8 = """
    select * from `emp`
    where 
    `JOB` in ("MANAGER", "SALESMAN")
"""
pd.DataFrame(
    db1.sql_query(query_8)
)

In [ ]:
# 사원의 이름이 특정 문자('S')로 시작하는 사원의 정보를 확인 
query_9 = """
    select * from `emp`
    where 
    `ENAME` like "s%%"
"""
pd.DataFrame(
    db1.sql_query(query_9)
)

In [ ]:
# emp_df에서 ENAME이 S로 시작하는 사원의 정보를 출력하자
# case1 
# 반복문 -> ENAME 컬럼의 데이터들을 하나씩 확인 
emp_df['ENAME']

word = 'S'
flag_list = [] 
for i in emp_df['ENAME']:
    # print(i[ : len(word)])
    # print( word.lower() == i[ : len(word)].lower())
    flag_list.append(
        # 특정 문자로 시작하는 경우의 조건
        word.lower() == i[ : len(word)].lower()
        # 특정 문자로 끝나는 경우의 조건
        # word.lower() == i[-len(word) : ].lower()
    )

emp_df.loc[flag_list, ]

In [ ]:
# 특정 문자가 포함되어있는가?
flag_list = []

word = 's'

for i in emp_df['ENAME']:
    flag_list.append(
        word.lower() in i.lower()
    )

emp_df.loc[flag_list, ]


In [ ]:
word = 's'
# word로 시작하는가?
emp_df['ENAME'].map(
    lambda x : x.lower().startswith(word)
)

In [ ]:
# word로 끝나는가?
emp_df['ENAME'].map(
    lambda x : x.lower().endswith(word)
)

In [ ]:
# 특정 문자 포함 여부
emp_df['ENAME'].str.lower().str.contains(word.lower())

### 데이터프레임의 결합 
- 유니언 결합 
    - 단순하게 행을 결합하거나 열을 결합하는 방식 
    - pandas에 내장된 concat() 함수를 이용
        - concat( [데이터프레임1, 데이터프레임2, ...] )
            - axis 매개변수
                - 0(index, rows) : 단순 행을 결합
                - 1(columns) : 단순 열을 결합 
            - ignore_index 매개변수
                - False : 결합되는 기준(인덱스, 컬럼)을 보존 
                - True : 결합되는 기준(인덱스, 컬럼)을 초기화
- 조인 결합 
    - 특정한 조건에 맞춰서 열을 결합 
    - 데이터프레임과 데이터프레임 간의 결합 -> 특정 컬럼의 데이터들이 같은 값들로 이루어져 있는 경우에 열을 추가하는 결합 방식
    - pandas, DataFrame에 내장된 merge() 함수 이용
    - 조건에 맞춰서 열을 결합 방식이기 때문에 concat()과는 다르게 2개의 데이터프레임만 결합 
        - pd.merge( 데이터프레임명, 데이터프레임명 ) // df.merge(데이터프레임명)
            - on 매개변수
                - 조건식(두 개의 데이터프레임이 공통적으로 가지고 있는 데이터 컬럼명)
                - 두개의 데이터프레임이 공통적으로 가지고 있는 데이터가 컬럼의 이름이 다른 경우 
                    - left_on 매개변수 첫번째 데이터프레임의 컬럼명
                    - right_on 매개변수 두번째 데이터프레임의 컬럼명
            - how 매개변수
                - 조인 결합의 방식을 지정 
                    - inner : 두 개의 데이터프레임이 공통적으로 가지고 있는 데이터만 결합하여 출력 
                    - left : 왼쪽(첫번째) 데이터프레임을 기준으로 결합 
                    - right : 오른쪽(두번째) 데이터프레임을 기준으로 결합 
                    - outer : 두개의 데이터프레임의 합집합

In [ ]:
# sql을 이용하여 tran_1, tran_2 테이블을 유니언 결합 
query_10 = """
    select * from `tran_1`
    union
    select * from `tran_2`
"""
pd.DataFrame(
    db1.sql_query(query_10)
)

In [ ]:
# 상대 경로 : 상위로 이동(../) -> csv 폴더 이동(csv/) -> tran_1.csv
tran_1 = pd.read_csv("../csv/tran_1.csv")
tran_2 = pd.read_csv("../csv/tran_2.csv")
tran_d_1 = pd.read_csv("../csv/tran_d_1.csv")
trad_d_2 = pd.read_csv("../csv/tran_d_2.csv")

In [ ]:
pd.concat( [tran_1, tran_2], axis=0 )

In [ ]:
# 조인 결합 
# tran_1, tran_d_1 조인 결합 
# 조건 확인 -> 두개의 데이터프레임이 가지고 있는 같은 데이터가 무엇인가?

In [ ]:
tran_1.head(1)

In [ ]:
tran_d_1.head(1)

In [ ]:
# 두개의 데이터프레임에서 공통적으로 가지고 잇는 데이터는 transaction_id
query_11 = """
    select 
    `t1`.`transaction_id` `거래 식별자`, 
    `payment_date` `지불 일자`, 
    `price` `가격`, 
    `quantity` `개수`, 
    `item_id` `아이템 이름`
    from
    `tran_1` `t1` left join `tran_d_1` `td1`
    on `t1`.`transaction_id` = `td1`.`transaction_id`
"""
pd.DataFrame(
    db1.sql_query(query_11)
)

In [ ]:
# 위의 결과와 같은 작업 
# left join 결합 
tran_df = pd.merge( tran_1, tran_d_1, 
         how = 'left', on= 'transaction_id' )

In [ ]:
# 특정 컬럼만 필터링 
# (transaction_id, payment_date, price, quantity, item_id)
tran_df.loc[ : , ['transaction_id', 'payment_date', 'price', 
                'quantity', 'item_id'] ]

In [ ]:
tran_df = tran_df[ ['transaction_id', 'payment_date', 'price', 
                'quantity', 'item_id'] ]

In [ ]:
# 컬럼의 이름을 변경 
# case1
# rename() : 인덱스를 재지정하거나 컬럼을 재지정
tran_df.rename(
    columns= {
        'transaction_id' : '거래 식별자', 
        'payment_date' : '지불 일자', 
        'price' : '가격', 
        'quantity' : '개수', 
        'item_id' : '아이템 이름'
    }
)

In [89]:

# case2
# 데이터프레임 안에 저장된 변수 columns를 변경 
# 바로 대입하는 경우 유의할 점 -> 
# 기존의 컬럼의 개수와 대입하려는 데이터의 개수가 같아야된다.
tran_df.columns = ['거래 식별자', '지불 일자', '가격', 
                   '개수', '아이템 이름']

In [90]:
tran_df

,거래 식별자,지불 일자,가격,개수,아이템 이름
0,T0000000113,2019-02-01 01:36:57,210000,1.0,S005
1,T0000000114,2019-02-01 01:37:23,50000,1.0,S001
2,T0000000115,2019-02-01 02:34:19,120000,1.0,S003
3,T0000000116,2019-02-01 02:47:23,210000,1.0,S005
4,T0000000117,2019-02-01 04:33:46,170000,2.0,S002
...,...,...,...,...,...
5238,T0000005108,2019-06-15 02:42:41,210000,NaN,NaN
5239,T0000005109,2019-06-15 03:36:16,150000,NaN,NaN
5240,T0000005110,2019-06-15 03:44:06,50000,NaN,NaN
5241,T0000005111,2019-06-15 04:14:06,210000,NaN,NaN


In [93]:
# 정렬 방식 
# DataFrame -> sort_values()
# SQL -> order by `컬럼` [ASC : 오름차순, DESC : 내림차순]

query_12 = """
    select * from `emp`
    where `JOB` = "MANAGER"
    order by `ENAME` desc
"""

pd.DataFrame(
    db1.sql_query(query_12)
)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0
1,7782.0,CLARK,MANAGER,7839.0,1981-06-09,2450.0,0.0,10.0
2,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0


In [97]:
# 그룹화 sql 
# JOB을 기준으로 그룹화 -> SAL의 평균을 구한다. 
# -> JOB, SAL의 평균 컬럼으로 지정 -> SAL의 평균을 기준으로 내림차순
query_13 = """
    select 
    `JOB`, AVG(`SAL`) `mean_SAL`
    from 
    `emp` 
    group by 
    `JOB`
    order by 
    `mean_SAL` DESC
"""

pd.DataFrame(
    db1.sql_query(query_13)
)


,JOB,mean_SAL
0,PRESIDENT,5000.000000
1,ANALYST,3000.000000
2,MANAGER,2758.333333
3,SALESMAN,1400.000000
4,CLERK,1037.500000
